<a href="https://colab.research.google.com/github/MadsHogenhaug/icao_scraper/blob/main/icao_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import time
from collections import Counter
from google.colab import files


In [ ]:
countries = ["Africa", "Antartica", "Asia","Europe","North_America", "Oceania","South_America"]
c = []

for i in countries:
  url = f"https://airportsbase.org/{i}"
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")

  div_element = soup.find("div", class_ = "table-wrap")
  table_rows = div_element.find_all("tr")

  for row in table_rows[1:]:
    columns = row.find_all("a")
    if columns:
        a = columns[0].text.strip()
        if a not in c:  # Check if the item is not already in the list
            c.append(a)
    else:
        print("empty")
  time.sleep(1)

c = [x.replace(" ", "_").replace(".", "").replace("(", "").replace(")", "").replace("'", "_") for x in sorted(c)]


In [ ]:
# Base URL for the table pages
airports = []

for i in c:
  base_url = f"https://airportsbase.org/{i}/all/airports"
  front_page_url = "https://airportsbase.org/"
  page_number = 1  # Start with t

  while True:
      # Construct the URL for the current page
      page_url = f"{base_url}/page{page_number - 1}"

      print(f"Crawling: {page_url}")  # Print the current website URL

      # Make a request to the page
      response = requests.get(page_url)

      if response.url == front_page_url:
          print("Redirected to front page. Exiting.")
          break

      soup = BeautifulSoup(response.content, "html.parser")

      a_elements = soup.find_all('th')  # Assuming the <a> elements are within <th> elements

      # Check if there are no <a> elements on the current page
      if not a_elements:
          print("No more <a> elements found. Exiting.")
          break

      # Loop through each <a> element
      for a_element in a_elements[8:]:
          a_text = a_element.get_text()  # Get the text within the <a> element
          td_element = a_element.find_next('td')  # Find the next <td> element
          td_text = td_element.get_text() if td_element else ""  # Get the text within the <td> element

          # print(f"Link Text: {a_text}")
          # print(f"Associated TD Text: {td_text}")
          # print("----------")
          airports.append({"Airport":a_text, "ICAO": td_text})
      # Move on to the next page
      page_number += 1
      time.sleep(1)

In [ ]:
for entry in airports:
    if not entry['ICAO']:
        entry['ICAO'] = 'NO ICAO'

df = pd.DataFrame(airports)

df.to_csv("icao_airports.csv",index=False,encoding="UTF-8")
files.download("icao_airports.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# abc = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# all_airline_data = []

# for i in abc:
#     url = f"https://airlinecodes.info/icao/{i}"
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")

#     div_element = soup.find("table", class_="tdata")

#     if div_element:
#         table_rows = div_element.find_all("tr")[1:]  # Skip the header row

#         for row in table_rows:
#             columns = row.find_all('td')
#             icao = columns[0].text.strip()
#             iata = columns[1].find('a').text.strip() if columns[1].find('a') else ''
#             name = columns[2].find('a').text.strip()
#             all_airline_data.append({'ICAO': icao, 'IATA': iata, 'NAME': name})

# df = pd.DataFrame(all_airline_data, columns=['ICAO', 'IATA', 'NAME'])
# print(df)

In [ ]:
# from google.colab import files

# df.to_csv("icao.csv",encoding="UTF-8")
# files.download("icao.csv")

